In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/toxicity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/identity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv


# **load train , test dataset**

In [3]:
train =pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
df_test = test['comment_text']
df = train[['target','comment_text']]

,target,comment_text
1460207,0.000000,"Spot On, This is the reason the Lame Stream Me..."
465767,0.500000,Holding Mr. Trump to higher standards in how h...
1126507,0.300000,Is there any evidence of this? Anything I rea...
7445,0.000000,When the weather gets warm near Thurston High ...
371766,0.166667,"Perhaps, Chinese should start to support Quebe..."


In [5]:
df.sample(5)

,target,comment_text
1660741,0.000000,"Um, Trump won't be meeting Hugo Chavez, unless..."
1232903,0.200000,You're the one who doesn't get it. You're say...
1781988,0.196721,"All form of hate need to be eliminated racism,..."
1787490,0.000000,"UK, recently after all the Murders committed b..."
1758924,0.200000,"An act of terrorism, as I said."


In [6]:
df.shape

(1804874, 2)

In [7]:
X = df['comment_text']
X.sample(5)

1596133    Well, I would start by no longer manufacturing...
1348647    History is history.  Wish they would not try t...
1742560    For years, white paint was slopped on the wind...
1530930    Here's the ADN article describing SB 91 from l...
1134362                SNAFU- Situation Normal All F--- UP ?
Name: comment_text, dtype: object

In [13]:
y = df['target']
y = np.where(y>=0.5,1.,0.)
y[:5]

array([0., 0., 0., 0., 1.])

In [51]:
unique, counts = np.unique(y, return_counts=True)
unique, counts

(array([0., 1.]), array([1660540,  144334]))

# Baseline model (using pipeline)

In [34]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=42)


In [36]:
neg_class_len = np.sum(y_train)
pos_class_len =df.shape[0] - int(neg_class_len)
print('negitive (1) class len is :',int(neg_class_len))
print('positive (0) class len is :',pos_class_len)

negitive (1) class len is : 100705
positive (0) class len is : 1704169


In [37]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(x_train, y_train);

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [38]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97    497834
         1.0       0.74      0.48      0.58     43629

    accuracy                           0.94    541463
   macro avg       0.85      0.73      0.78    541463
weighted avg       0.94      0.94      0.94    541463

accuracy: 0.945


# look inside the model

In [39]:
import eli5
eli5.show_weights(clf, vec=vec, top=20)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+7.024,idiotic
+6.845,idiots
+6.260,stupidity
+6.217,idiot
+6.112,morons
+6.033,shit
+6.012,moron
+5.794,hypocrite
+5.590,stupid
+4.953,scum


In [42]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf, row['comment_text'], vec=vec,))
    print("--"*50)

true label: 0.0


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.1666666666666666


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.6


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------


# Try Tfidf with some processing
 **using char level and ngrams **

In [43]:
vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.5)
clf = LinearSVC()
pipe_tfidf = make_pipeline(vec, clf)
pipe_tfidf.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('linearsvc', LinearSVC())])

In [44]:
print_report(pipe_tfidf, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97    497834
         1.0       0.81      0.35      0.49     43629

    accuracy                           0.94    541463
   macro avg       0.88      0.67      0.73    541463
weighted avg       0.93      0.94      0.93    541463

accuracy: 0.941


In [45]:
eli5.show_weights(clf, vec=vec, top=20)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+7.835,hit
+6.854,idi
+6.810,iot
+5.881,crit
+5.725,umb
+5.394,tupid
+5.356,ffo
+5.216,oran
+5.207,owar
+5.202,side


In [46]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf, row['comment_text'], vec=vec,))
    print("--"*50)

true label: 0.0


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------


# It seem that was a big imbalance in the dataset which make overfitting so lets solve the imbalance problem in 02- classic nlp sloution notebook